In [50]:
import numpy as np
import openpyxl
import polars as pl

In [51]:
ruta = "data/opiniones.xlsx"
workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames
df_excel = pl.read_excel(ruta, sheet_id=2).lazy()

In [52]:

def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index]
    print(f"Nombre de la hoja: {nombre_hoja}")
    return nombre_hoja


def añadir_columna_lugar_turistico(df_excel: pl.LazyFrame, iteracion: int) -> pl.LazyFrame:
    nombre_hoja = obtener_nombre_hoja(iteracion)
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico")
        ])
    )
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col

In [53]:
lista_df: list[pl.LazyFrame] = []
for i, hoja in enumerate(hojas):
    df_excel = añadir_columna_lugar_turistico(df_excel, i)
    lista_df.append(df_excel)
    
df_excel = juntar_df(lista_df)


df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.columns})

Nombre de la hoja: Basilica Colegiata 560
Nombre de la hoja: Mercado Hidalgo 600
Nombre de la hoja: Casa de Diego Rivera 698
Nombre de la hoja: Universidad de Guanajuato 900
Nombre de la hoja: Alhóndiga 930
Nombre de la hoja: Teatro Juárez 1,010
Nombre de la hoja: Jardín de la Unión 1,134
Nombre de la hoja: Callejón del Beso 1,360
Nombre de la hoja: Monumento Pípila 1,620
Nombre de la hoja: Museo de las Momias 1,650


/var/folders/j4/21ypxx3x0019wgj7rqnlm2340000gn/T/ipykernel_64950/3389233938.py:9: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.columns})


In [54]:
df_excel = df_excel.collect()

print(df_excel)


# Guardar el DataFrame en un parquet
df_excel.write_parquet("data/opiniones_lugar_turistico.parquet")
print("Archivo guardado en: data/opiniones_lugar_turistico.parquet")

shape: (6_000, 13)
┌───────────┬──────┬─────────────┬────────────┬───┬─────────┬────────────┬────────────┬────────────┐
│ género    ┆ edad ┆ nacional_ó_ ┆ calificaci ┆ … ┆ idioma  ┆ dispositiv ┆ fecha      ┆ lugar_turi │
│ ---       ┆ ---  ┆ internacion ┆ ón         ┆   ┆ ---     ┆ o          ┆ ---        ┆ stico      │
│ str       ┆ i64  ┆ al          ┆ ---        ┆   ┆ str     ┆ ---        ┆ date       ┆ ---        │
│           ┆      ┆ ---         ┆ str        ┆   ┆         ┆ str        ┆            ┆ str        │
│           ┆      ┆ str         ┆            ┆   ┆         ┆            ┆            ┆            │
╞═══════════╪══════╪═════════════╪════════════╪═══╪═════════╪════════════╪════════════╪════════════╡
│ Femenino  ┆ 33   ┆ Internacion ┆ Excelente  ┆ … ┆ Inglés  ┆ Móvil      ┆ 2018-04-20 ┆ Basilica   │
│           ┆      ┆ al          ┆            ┆   ┆         ┆            ┆            ┆ Colegiata  │
│           ┆      ┆             ┆            ┆   ┆         ┆           